In [ ]:
import numpy as np
from mpi4py import MPI
import time
import pathlib
from IPython import display
import matplotlib.pyplot as plt

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)


#Ras = np.linspace(1e6, 1e7, 500)
#Ras = [format(x, '.3e') for x in Ras] 
#string = [str(x) for x in Ras]
#string = [x.replace('.', 'p') for x in string]
#string = [x.replace('+', '') for x in string]
#string = [(str(i) + '_' + x) for (i, x) in zip(range(1000), string)]

#Ras = np.linspace(1e7, 1e8, 800)
#Ras = [format(x, '.3e') for x in Ras]
#Ras = Ras[1:]

#string = [str(x) for x in Ras]
#string = [x.replace('.', 'p') for x in string]
#string = [x.replace('+', '') for x in string]
#string = [(str(i + 500) + '_' + x) for (i, x) in zip(range(800), string)]

Ras = np.linspace(1e8, 1e9, 1000)
Ras = [format(x, '.3e') for x in Ras]
Ras = Ras[1:]

string = [str(x) for x in Ras]
string = [x.replace('.', 'p') for x in string]
string = [x.replace('+', '') for x in string]
string = [(str(i + 1299) + '_' + x) for (i, x) in zip(range(1000), string)]


ss = 900
ee = 920

Ras = Ras[ss:ee]
string = string[ss:ee]

for i in range(len(Ras)):

    # Parameters
    Lx, Lz = (1, 1)
    Prandtl = 1.
    Rayleigh = float(Ras[i])
    delta_T = 1
    T_b = delta_T/2
    T_t = -delta_T/2

    # Create bases and domain
    x_basis = de.Fourier('x', 256, interval=(0, Lx), dealias=3/2)
    z_basis = de.Chebyshev('z', 256, interval=(-Lz/2, Lz/2), dealias=3/2)
    domain = de.Domain([x_basis, z_basis], grid_dtype=np.float64)

    # 2D Boussinesq hydrodynamics
    problem = de.IVP(domain, variables=['p','T','u','w','Tz','uz','wz'])
    problem.parameters['P'] = (Rayleigh * Prandtl)**(-1/2)
    problem.parameters['R'] = (Rayleigh / Prandtl)**(-1/2)
    problem.parameters['T_b'] = T_b
    problem.parameters['T_t'] = T_t

    problem.add_equation("dx(u) + wz = 0")
    problem.add_equation("dt(T) - P*(dx(dx(T)) + dz(Tz))             = -(u*dx(T) + w*Tz)")
    problem.add_equation("dt(u) - R*(dx(dx(u)) + dz(uz)) + dx(p)     = -(u*dx(u) + w*uz)")
    problem.add_equation("dt(w) - R*(dx(dx(w)) + dz(wz)) + dz(p) - T = -(u*dx(w) + w*wz)")

    problem.add_equation("Tz - dz(T) = 0")
    problem.add_equation("uz - dz(u) = 0")
    problem.add_equation("wz - dz(w) = 0")

    problem.add_bc("left(T) = T_b")
    problem.add_bc("left(u) = 0")
    problem.add_bc("left(w) = 0")
    problem.add_bc("right(T) = T_t")
    problem.add_bc("right(u) = 0")
    problem.add_bc("right(w) = 0", condition="(nx != 0)")
    problem.add_bc("integ(p) = 0", condition="(nx == 0)")

    # Build solver
    solver = problem.build_solver(de.timesteppers.RK222)
    logger.info('Solver built')

    # Initial conditions or restart
    if not pathlib.Path('restart.h5').exists():

        # Initial conditions
        x, z = domain.all_grids()
        T = solver.state['T']
        Tz = solver.state['Tz']

        # Random perturbations, initialized globally for same results in parallel
        gshape = domain.dist.grid_layout.global_shape(scales=1)
        slices = domain.dist.grid_layout.slices(scales=1)
        rand = np.random.RandomState(seed=42)
        noise = rand.standard_normal(gshape)[slices]

        # Linear background + perturbations damped at walls
        zb, zt = z_basis.interval
        pert =  1e-3 * noise * (zt - z) * (z - zb)
        T['g'] = pert +  T_b + ((z + 0.5*Lz)/Lz)*(T_t - T_b)
        T.differentiate('z', out=Tz)

        # Timestepping and output
        dt = 0.1
        stop_sim_time = 20
        fh_mode = 'overwrite'
    else:
        # Restart
        write, last_dt = solver.load_state('restart.h5', -1)

        # Timestepping and output
        dt = last_dt
        stop_sim_time = 20
        fh_mode = 'append'

    # Integration parameters
    solver.stop_sim_time = stop_sim_time

    snapshots = solver.evaluator.add_file_handler('snapshots_' + string[i], sim_dt=0.1, mode=fh_mode)
    snapshots.add_system(solver.state)

    # CFL
    #CFL = flow_tools.CFL(solver, initial_dt=dt, cadence=10, safety=0.5,
                   # max_change=1.5, min_change=0.5, max_dt=0.125, threshold=0.05)
    #CFL.add_velocities(('u', 'w'))

    # Flow properties
    flow = flow_tools.GlobalFlowProperty(solver, cadence=10)
    flow.add_property("sqrt(u*u + w*w) / R", name='Re')

    # Main loop
    try:
        logger.info('Starting loop')
        start_time = time.time()
        while solver.proceed:

            if solver.sim_time >= 0 and solver.sim_time < 5:
                dt = 0.1
            elif solver.sim_time >= 5 and solver.sim_time < 10:
                dt = 0.01
            else:
                dt = 0.001

            #dt = CFL.compute_dt()
            dt = solver.step(dt)
            if (solver.iteration-1) % 10 == 0:
                logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))
                logger.info('Max Re = %f' %flow.max('Re'))
    except:
        logger.error('Exception raised, triggering end of main loop.')
        raise
    finally:
        end_time = time.time()
        logger.info('Iterations: %i' %solver.iteration)
        logger.info('Sim end time: %f' %solver.sim_time)
        logger.info('Run time: %.2f sec' %(end_time-start_time))
        logger.info('Run time: %f cpu-hr' %((end_time-start_time)/60/60*domain.dist.comm_cart.size))